# Notes

## HMM POS
HMM is a sequence model, assigns a label to each unit in a sequence. Maps sequence of obsvs to a sequence of labels. It is a probablistic model, given a sequence it computes a probability distr over possible sequence of labels and chooses the label sequence with the highest probability.

### Markov Chains
Tells us something about the probabilities of the sequence of random variables (states). Assumes that future predictions only depend on the current state, and not the past.

Markov Assumption: P(qi = a|q1...qi−1) = P(qi = a|qi−1) 

The value of transition probabilities from a state must sum to one

### Hidden Markov Model
Markov chains are used when the sequence is observable. HMM predict sequences that are hidden (unobserved)
HMM allows us to talk about observed events and hidden events which we think of as causal factors in the model.

Q = set of states. A = transition probability matrix (aij is prob oving i to j)
O = sequence of observations. B = sequence of observation likelihoods expressing prob of oi being generated from a state qi, PI = initial prob distr of starting states.

Output Independence: P(oi
|q1 ...qi
,...,qT ,o1,...,oi
,...,oT ) = P(oi
|qi)

HMM has A and B probability components. A matrix contains the tag transition probs. prob of a tag occuring after prev tag.

<img src=imgs/Amatrix.png>

B emision probs P(wi|ti) repr the prob given a tag, it wil be associated with a given word.

<img src=imgs/Bmatrix.png>

<img src=imgs/assum.png>

<img src=imgs/final.png>



# Vertibi Algorithm
Decoding algorithm for HMMs is the Viterbi Algorithm. Start with a prob lattice of with N columns (n is num words in sentence) and t rows. (t is the number of states in the state graph... POS)

Each cell of the lattice v(j) represents the probability that the HMM is in state j after seeing the first t observations and passing through the most probable state sequence q1....qt-1. At each cell, recursively compute the most probable path that would lead to that cell.

<img src=imgs/vit.png>



In [16]:
import os
import io
import sys

class Tagger:
    def __init__(self):
        self.initial_tag_probability = None
        self.transition_probability = None
        self.emission_probability = None
        self.suffix_probability = None

    def load_corpus(self, path):
        if not os.path.isdir(path):
            sys.exit("Input path is not a directory")
        for filename in os.listdir(path):
            filename = os.path.join(path, filename)
            try:
                reader = io.open(filename)
                data = reader.readlines()
                
                #remove header and footer
                data = data[2:-2]

                #replace utf-16 formatting
                #data = [sentence.replace("\n",'') for sentence in data]
                data = [sentence.replace("\n",'') for sentence in data]
                data = [sentence.replace("\00",'') for sentence in data]

                #split sentence into list on spaces
                data = [sentence.split() for sentence in data]

                output = []
                for innerlist in data:
                    #skip sentences of len < 2
                    if len(innerlist)<3:
                        continue
                    inneroutput = []
                    for pair in innerlist:
                        pair = pair.split('/')
                        #remove PAIRs that dont split in two
                        if len(pair)!=2:
                            continue
                        #Getting rid of these suckers
                        if pair[0]=='brown_modified':
                            continue
                        else:
                            inneroutput.append( (pair[0].lower(),pair[1]) )
                    #dont include sentences with 1 or fewer pairs
                    if len(inneroutput)<2:
                        continue
                    output.append(inneroutput)  
                
                return output
            except IOError:
                sys.exit("Cannot read file")

    def initialize_probabilities(self, sentences):
        if type(sentences) != list:
            sys.exit("Incorrect input to method")
        
        def ifin_add(var, dic):
            #Helper function to count
            if var in dic:
                dic[var] += 1
            else:
                dic[var] = 1

        def ifin_div(var, dic, divisor, v_size):
            #Helper function to divide WITH SMOOTHING
            if var in dic:
                dic[var] = (dic[var] + 1) / (divisor + v_size)
            else:
                dic[var] = 1 / (divisor + v_size)

        TAG = {}
        WORD = {}
        TAG2 = {}
        INIT = {}
        SUFFIX = {}
        for sentence in sentences:
            for i in range(len(sentence)):
                word, tag = sentence[i][0], sentence[i][1]
                #count initial tag of sentence
                if i==0:
                    ifin_add(tag, INIT)

                #Tag counts for all sentences C(ti)
                ifin_add(tag, TAG)

                #Tag_counts for word-tag C(ti,wi)
                if word not in WORD:
                    WORD[word]={}
                ifin_add(tag, WORD[word])

                suffix = word[-2:]
                #Tag_counts for suffix-tag C(ti,wi-2)
                if suffix not in SUFFIX:
                    SUFFIX[suffix]={}
                ifin_add(tag, SUFFIX[suffix])

                #tag-tag counts C(ti,ti-1)
                if i > 0:
                    to_tag = tag
                    from_tag = sentence[i-1][1]
                    if from_tag not in TAG2:
                        TAG2[from_tag]={}
                    ifin_add(to_tag, TAG2[from_tag])
        
        #Convert counts to probabilities
        tags = list(TAG.keys())
        words = list(WORD.keys())
        suffixes = list(SUFFIX.keys())
        vocab_size = len(words)
        for tag in tags:
            #initial probabilities
            ifin_div(tag, INIT, len(sentences), vocab_size)

            #emission probabilities
            for word in words:
                ifin_div(tag, WORD[word], TAG[tag], vocab_size)
            #suffix probabilities
            for suff in suffixes:
                ifin_div(tag, SUFFIX[suff], TAG[tag], len(suffixes))

            #transition probabilities
            for to_tag in tags:
                ifin_div(to_tag, TAG2[tag], TAG[tag], vocab_size)       
        
        self.initial_tag_probability = INIT
        self.transition_probability = TAG2
        self.emission_probability = WORD
        self.suffix_probability = SUFFIX
        return

    def viterbi_decode(self, sentence):
        if type(sentence) != str:
            sys.exit("Incorrect input to method")
            
        sentence = sentence.lower().split()

        tags = list(self.initial_tag_probability.keys())
        
        #List of dictionaries so i dont have to remember which POS is which index
        viterb = [{tag:j for (tag,j) in zip(tags,range(len(tags)))} for i in range(len(sentence))]
        backp = [{tag:j for (tag,j) in zip(tags,range(len(tags)))} for i in range(len(sentence))]

        #Calculate the self.initial_tag_probabilityial probabilities
        for tag in tags:
            if sentence[0] in self.emission_probability:
                viterb[0][tag] = self.initial_tag_probability[tag]*self.emission_probability[sentence[0]][tag]
            else:
                viterb[0][tag] = 0.0
            backp[0][tag] = 0

        #Calculating the lattice    
        for t in range(1, len(sentence)):
            for tag in tags:
                #The backwards probability for POS = P(t-1,s')*P(s'->s)*P(w,s)
                if sentence[t] in WORD:
                    vals = {prev_tag : viterb[t-1][prev_tag] * TAG2[prev_tag][tag] * WORD[sentence[t]][tag] for prev_tag in tags}
                else:
                    vals = {prev_tag : viterb[t-1][prev_tag] * TAG2[prev_tag][tag] * SUFFIX[sentence[t][-2:]][tag] for prev_tag in tags}
                viterb[t][tag] = vals[max(vals, key=vals.get)]
                backp[t][tag] = max(vals, key=vals.get)
        
        #extracting highest value POS
        prob_seq = []
        t = len(sentence)-1
        best_backpath = max(viterb[t], key=viterb[t].get)
        prob_seq.append(best_backpath)
        for i in range(0, len(sentence)-1):
            t=len(sentence)-i-1
            best_backpath = backp[t][best_backpath]
            prob_seq.append(best_backpath)
        return prob_seq[::-1]

In [17]:
path = 'C:\\Users\\harri\\Desktop\\NLP Assignment 2\\brown_modified_pos'
tag = Tagger()
sentences = tag.load_corpus(path)
tag.initialize_probabilities(sentences)
sentence = "People continue to enquire the reason for the race for outer space ."
tag.viterbi_decode(sentence)

['NOUN',
 'VERB',
 'X',
 'VERB',
 'DETERMINER',
 'NOUN',
 'PREPOSITION',
 'DETERMINER',
 'NOUN',
 'PREPOSITION',
 'ADJECTIVE',
 'NOUN',
 'PUNCT']

# Testing Zone

In [10]:
path = 'C:\\Users\\harri\\Desktop\\NLP Assignment 2\\brown_modified_pos\\brown_modified_pos'
reader = io.open(path, encoding='utf-8')
data = reader.readlines()
#remove header and footer
data = data[2:-2]

#replace utf-16 formatting
data = [sentence.replace("\n",'') for sentence in data]
data = [sentence.replace("\00",'') for sentence in data]


#split sentence into list on spaces
data = [sentence.split() for sentence in data]

output = []
for innerlist in data:
    #skip sentences of len < 2
    if len(innerlist)<3:
        continue
    inneroutput = []
    for pair in innerlist:
        pair = pair.split('/')
        #remove PAIRs that dont split in two
        if len(pair)!=2:
            continue
        if pair[0]=='brown_modified':
            continue
        else:
            inneroutput.append( (pair[0].lower(),pair[1]) )
    #dont include sentences with 1 or fewer pairs
    if len(inneroutput)<2:
        continue
    output.append(inneroutput)    
print(output[6])

[('finally', 'ADVERB'), (',', 'PUNCT'), ('it', 'PRONOUN'), ('did', 'VERB'), ('seem', 'VERB'), ('clear', 'ADJECTIVE'), ('as', 'CONJUNCTION'), ('day', 'NOUN'), ('to', 'PREPOSITION'), ('these', 'DETERMINER'), ('clergymen', 'NOUN'), (',', 'PUNCT'), ('as', 'CONJUNCTION'), ("gannett's", 'NOUN'), ('son', 'NOUN'), ('explained', 'VERB'), ('in', 'PREPOSITION'), ('the', 'DETERMINER'), ('biography', 'NOUN'), ('of', 'PREPOSITION'), ('his', 'PRONOUN'), ('father', 'NOUN'), (',', 'PUNCT'), ('they', 'PRONOUN'), ('had', 'VERB'), ('always', 'ADVERB'), ('contended', 'VERB'), ('for', 'PREPOSITION'), ('the', 'DETERMINER'), ('propriety', 'NOUN'), ('of', 'PREPOSITION'), ('their', 'PRONOUN'), ('claim', 'NOUN'), ('to', 'PREPOSITION'), ('the', 'DETERMINER'), ('title', 'NOUN'), ('of', 'PREPOSITION'), ('christians', 'NOUN'), ('.', 'PUNCT')]


In [11]:
'tacos'[-1:]

's'

In [12]:
sentences = output
def ifin_add(var, dic):
    #Helper function to count
    if var in dic:
        dic[var] += 1
    else:
        dic[var] = 1

def ifin_div(var, dic, divisor, v_size):
    #Helper function to divide WITH SMOOTHING
    if var in dic:
        dic[var] = (dic[var] + 1) / (divisor + v_size)
    else:
        dic[var] = 1 / (divisor + v_size)
        
TAG = {}
WORD = {}
TAG2 = {}
INIT = {}
SUFFIX = {}
for sentence in sentences:
    for i in range(len(sentence)):
        word, tag = sentence[i][0], sentence[i][1]
        #count initial tag of sentence
        if i==0:
            ifin_add(tag, INIT)
        
        #Tag counts for all sentences C(ti)
        ifin_add(tag, TAG)
        
        #Tag_counts for word-tag C(ti,wi)
        if word not in WORD:
            WORD[word]={}
        ifin_add(tag, WORD[word])
        
        suffix = word[-2:]
        #Tag_counts for suffix-tag C(ti,wi-2)
        if suffix not in SUFFIX:
            SUFFIX[suffix]={}
        ifin_add(tag, SUFFIX[suffix])
        
        #tag-tag counts C(ti,ti-1)
        if i > 0:
            to_tag = tag
            from_tag = sentence[i-1][1]
            if from_tag not in TAG2:
                TAG2[from_tag]={}
            ifin_add(to_tag, TAG2[from_tag])

tags = list(TAG.keys())
words = list(WORD.keys())
suffixes = list(SUFFIX.keys())
vocab_size = len(words)
#Convert counts to probabilities
for tag in tags:
    #initial probabilities
    ifin_div(tag, INIT, len(sentences), vocab_size)
        
    #emission probabilities
    for word in words:
        ifin_div(tag, WORD[word], TAG[tag], vocab_size)
        
    for suff in suffixes:
        ifin_div(tag, SUFFIX[suff], TAG[tag], len(suffixes))
    
    #transition probabilities
    for to_tag in tags:
        ifin_div(to_tag, TAG2[tag], TAG[tag], vocab_size)

In [15]:
sentence = 'Finally , she enquires the day .'
sentence = sentence.lower().split()

#List of dictionaries so i dont have to remember which POS is which index
viterb = [{tag:j for (tag,j) in zip(tags,range(len(tags)))} for i in range(len(sentence))]
backp = [{tag:j for (tag,j) in zip(tags,range(len(tags)))} for i in range(len(sentence))]

#Calculate the initial probabilities
for tag in tags:
    if sentence[0] in WORD:
        viterb[0][tag] = INIT[tag]*WORD[sentence[0]][tag]
    else:
        viterb[0][tag] = 0.0
    backp[0][tag] = 0
    
#Calculating the lattice    
for t in range(1, len(sentence)):
    for tag in tags:
        #The backwards probability for POS = P(t-1,s')*P(s'->s)*P(w,s)
        if sentence[t] in WORD:
            vals = {prev_tag : viterb[t-1][prev_tag] * TAG2[prev_tag][tag] * WORD[sentence[t]][tag] for prev_tag in tags}
        else:
            vals = {prev_tag : viterb[t-1][prev_tag] * TAG2[prev_tag][tag] * SUFFIX[sentence[t][-2:]][tag] for prev_tag in tags}
        viterb[t][tag] = vals[max(vals, key=vals.get)]
        backp[t][tag] = max(vals, key=vals.get)

prob_seq = []
t = len(sentence)-1
best_backpath = max(viterb[t], key=viterb[t].get)
prob_seq.append(best_backpath)
for i in range(0, len(sentence)-1):
    t=len(sentence)-i-1
    best_backpath = backp[t][best_backpath]
    prob_seq.append(best_backpath)
prob_seq = prob_seq[::-1]
print(prob_seq)

['ADVERB', 'PUNCT', 'PRONOUN', 'VERB', 'DETERMINER', 'NOUN', 'PUNCT']
